<a href="https://colab.research.google.com/github/Prad0510/explainable-ews-finrisk/blob/main/ews_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
df = pd.read_csv("train.csv")

In [35]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [36]:
df.shape

(30000, 25)

In [37]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

In [38]:
ews_cols = [ 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month']
df_ews = df[ews_cols].copy()

In [39]:
df_ews.isnull().sum()

,0
PAY_0,0
PAY_2,0
PAY_3,0
PAY_4,0
PAY_5,0
PAY_6,0
BILL_AMT1,0
BILL_AMT2,0
BILL_AMT3,0
BILL_AMT4,0


Add risk trend feature to further generate a weighted behavioural trend. Recent payment delay are considered as stronger signal whereas older delays as weaker influence.

In [40]:
df_ews["risk_trend_score"] = (
    0.30 * df_ews["PAY_0"] +
    0.25 * df_ews["PAY_2"] +
    0.20 * df_ews["PAY_3"] +
    0.15 * df_ews["PAY_4"] +
    0.10 * df_ews["PAY_5"]
)

In [41]:
df_ews[["PAY_0","PAY_2","risk_trend_score"]].head()

,PAY_0,PAY_2,risk_trend_score
0,2,2,0.55
1,-1,2,0.20
2,0,0,0.00
3,0,0,0.00
4,-1,0,-0.50


Create a feature to track the fluctuation in the payment behaviour. We use standard deviation across payment history. Stable payer will have low volitality and rapidly changing behaviour will have high volitality.

In [42]:
pay_cols = ["PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6"]
df_ews["payment_volatility"] = df_ews[pay_cols].std(axis=1)

In [43]:
df_ews[["risk_trend_score","payment_volatility"]].head(10)

,risk_trend_score,payment_volatility
0,0.55,1.861899
1,0.20,1.224745
2,0.00,0.000000
3,0.00,0.000000
4,-0.50,0.516398
5,0.00,0.000000
6,0.00,0.000000
7,-0.45,0.547723
8,0.40,0.816497
9,-1.90,0.516398


In [44]:
bill_cols = ["BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6"]
pay_amt_cols = ["PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6"]

df_ews["total_bill"] = df_ews[bill_cols].sum(axis=1)
df_ews["total_paymemt"] = df_ews[pay_amt_cols].sum(axis=1)

Create a Liquidity Stress Feature to look for borrowers who owe more than they repay.

In [45]:
df_ews["liquidity_stress"] = (
    (df_ews["total_bill"] - df_ews["total_paymemt"])
    / (df_ews["total_bill"].abs() + 1)
)

In [46]:
df_ews[["risk_trend_score","payment_volatility","liquidity_stress"]].head(10)

,risk_trend_score,payment_volatility,liquidity_stress
0,0.55,1.861899,0.910448
1,0.20,1.224745,0.707167
2,0.00,0.000000,0.891603
3,0.00,0.000000,0.963737
4,-0.50,0.516398,0.459941
5,0.00,0.000000,0.967356
6,0.00,0.000000,0.933656
7,-0.45,0.547723,0.644695
8,0.40,0.816497,0.896309
9,-1.90,0.516398,0.475111


In [47]:
def minmax(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-6)


In [48]:
df_ews["trend_norm"] = minmax(df_ews["risk_trend_score"])
df_ews["volatility_norm"] = minmax(df_ews["payment_volatility"])
df_ews["liquidity_norm"] = 1 - minmax(df_ews["liquidity_stress"])


In [49]:
df_ews["ews_risk_score"] = (
    0.45 * df_ews["trend_norm"] +
    0.30 * df_ews["volatility_norm"] +
    0.25 * df_ews["liquidity_norm"]
)

In [51]:
print(df_ews[[
    "risk_trend_score",
    "payment_volatility",
    "liquidity_stress",
    "ews_risk_score"
]].head(10))

   risk_trend_score  payment_volatility  liquidity_stress  ews_risk_score
0              0.55            1.861899          0.910448        0.309326
1              0.20            1.224745          0.707167        0.231142
2              0.00            0.000000          0.891603        0.105883
3              0.00            0.000000          0.963737        0.105882
4             -0.50            0.516398          0.459941        0.127762
5              0.00            0.000000          0.967356        0.105882
6              0.00            0.000000          0.933656        0.105882
7             -0.45            0.547723          0.644695        0.133342
8              0.40            0.816497          0.896309        0.203506
9             -1.90            0.516398          0.475111        0.053644
